<a href="https://colab.research.google.com/github/VladLoPG/Lobanov-Programming-HSE/blob/main/deep-learning/Lobanov_summary_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytubefix ffmpeg-python git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip uninstall -y langchain langchain-core langchain-groq langchain-community

!pip install -qU langchain langchain-groq langchain-community langchain-text-splitters langchain-core

Found existing installation: langchain 1.0.0
Uninstalling langchain-1.0.0:
  Successfully uninstalled langchain-1.0.0
Found existing installation: langchain-core 1.0.0
Uninstalling langchain-core-1.0.0:
  Successfully uninstalled langchain-core-1.0.0
Found existing installation: langchain-groq 1.0.0
Uninstalling langchain-groq-1.0.0:
  Successfully uninstalled langchain-groq-1.0.0
Found existing installation: langchain-community 0.4
Uninstalling langchain-community-0.4:
  Successfully uninstalled langchain-community-0.4


In [ ]:
import whisper

In [ ]:
from pytubefix import YouTube
from pytubefix.cli import on_progress

def download_audio(youtube_url, output_path='audio.mp3'):
    yt = YouTube(youtube_url, on_progress_callback=on_progress)
    audio_stream = yt.streams.filter(only_audio=True).first()
    audio_stream.download(filename=output_path)
    print(f'Audio downloaded successfully: {output_path}')
    return output_path

# видео 30 мин на рус про лин регрессию
# youtube_url = 'https://www.youtube.com/watch?v=KJA9A1q9l7E'

# шортс про докер на рус
youtube_url = 'https://www.youtube.com/shorts/ui6ISsbrSac'

# топлес про угадывание чисел
# youtube_url = 'https://www.youtube.com/watch?v=f-D4Z0NMsb0&t=590s'

download_audio(youtube_url)

Audio downloaded successfully: audio.mp3


'audio.mp3'

In [ ]:
import whisper

def transcribe_audio(audio_file):
    model = whisper.load_model("medium")
    result = model.transcribe(audio_file)
    return result['text']

audio_file = 'audio.mp3'
transcription = transcribe_audio(audio_file)

In [ ]:
# with open('transcript_linear_whispermedium.txt') as file:
#  data = file.read()

**Задание:** написать модель для суммаризации текста видео с помощью LangChain.

[Туториал](https://python.langchain.com/v0.2/docs/tutorials/summarization/)

- создать цепочку с промптом "суммаризируй текст"

- вывести саммари

In [ ]:
import getpass
import os

from pytubefix import YouTube
from pytubefix.cli import on_progress
import whisper

from langchain_classic.chains.summarize import load_summarize_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.chat_models import init_chat_model


class BasicVideoSummarizer():
  '''
  Класс кратко излагает содержание видео с ютуба в текстовом виде
  1. Загружает видео и извлекает аудио
  2. Транскрибирует аудио с помощью whisper
  3. Чанкирует и преобразует в Document
  4. Суммаризирует с помощью LLM

  Attributes:
    whisper_model: Whisper Medium
    llm: Llama 4 Maverick 17b 128e instruct, Groq API
    text_splitter: RecursiveCharacterTextSplitter
    chain: load_summarize_chain

  Methods:
    run(video_url: str) -> str: Возвращает саммари видео по переданной ссылке с ютуба
  '''

  def __init__(self):
      self.whisper_model = whisper.load_model("medium") # качество хорошее и грузится быстрее большой модели
      self.llm = init_chat_model("meta-llama/llama-4-maverick-17b-128e-instruct", model_provider="groq") # вроде дает неплохие результаты и с гроком удобно работать
      self.text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000, chunk_overlap=300, add_start_index=True
) # для более длинных видео лучше порезать текст
      self.chain = load_summarize_chain(self.llm, chain_type="map_reduce")

  def _download_audio(self, video_url, output_path='audio.mp3'):
    try:
      yt = YouTube(video_url, on_progress_callback=on_progress)
      audio_stream = yt.streams.filter(only_audio=True).first()
      audio_stream.download(filename=output_path)
      print(f'Аудио загружено успешно!')
      return output_path
    except Exception as e:
      print(f'Произошла ошибка при загрузке видео: {e}')

  def _transcribe_audio(self, output_path):
    try:
      result = self.whisper_model.transcribe(output_path)
      text = result['text']
      print('Транскрибация прошла успешно!')
      return text
    except Exception as e:
      print(f'Произошла ошибка при транскрибации: {e}')

  def _chunkize(self, text):
    try:
      doc = Document(page_content=text)
      docs_split = self.text_splitter.split_documents([doc])
      print('Чанкирование прошло успешно!')
      return docs_split
    except Exception as e:
      print(f'Произошла ошибка при чанкировании: {e}')


  def run(self, video_url: str) -> str:
      '''
      Метод последовательно запускает рабочие функции и возвращает саммари в виде текстовой строки
      '''
      audio = self._download_audio(video_url)
      transcription = self._transcribe_audio(audio)
      chunks = self._chunkize(transcription)
      result = self.chain.run(chunks)
      print('Успех, все получилось без ошибок, можешь принтить результат')
      return result



In [ ]:
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

Enter API key for Groq: ··········


In [ ]:
summarizer = BasicVideoSummarizer()
summary = summarizer.run('https://www.youtube.com/watch?v=WFbPcA9aA2o&t=938s')

Аудио загружено успешно!
Транскрибация прошла успешно!
Чанкирование прошло успешно!
Успех, все получилось без ошибок, можешь принтить результат


In [ ]:
# читаемо выводит результат
from IPython.display import display, Markdown
display(Markdown(summary))

The author complains about the pervasive presence of advertisements in daily life, citing examples on YouTube, TikTok, smart appliances, and personal devices. They criticize companies like Samsung and YouTube for increasing ads, including retroactive ads on older content, and for taking a larger share of ad revenue, harming creators. The author also exposes fake social media influencers promoting products, such as "Mind Flow AI", without disclosing sponsorships, potentially violating FTC regulations. They lament the trend of prioritizing profits over user experience, introducing more ads, and emphasize the need for transparency in sponsored content. The author concludes by reflecting on the overwhelming digital stimulation and suggesting a return to nature as a potential solution.